In [95]:
import requests
import os
import yaml
import pandas as pd
import numpy as np

- Import credentials

In [96]:
credentials = yaml.load(open(os.path.expanduser('~/.ssh/lending_club_api.yml')))

In [97]:
account_id = credentials['account_id']

- Make the API call

In [98]:

r = requests.get('https://api.lendingclub.com/api/investor/v1/loans/listing',
                 headers={'Authorization': credentials['authentication'],'Content-Type': 'application/json',
                          'Accept': 'application/json'}, params = {'showAll':'true'})

In [99]:
r

<Response [200]>

In [100]:
json_response =  r.json()

In [127]:
json_response.keys()

[u'loans', u'asOfDate']

In [128]:
test_loan = json_response['loans'][50]

In [129]:
test_loan['purpose']

u'debt_consolidation'

- Convert to DF

In [130]:
loans_df = pd.DataFrame(json_response['loans'])

In [131]:
loans_df.head()

,accNowDelinq,accOpenPast24Mths,acceptD,addrState,addrZip,allUtil,annualInc,annualIncJoint,applicationType,avgCurBal,...,totCollAmt,totCurBal,totHiCredLim,totalAcc,totalBalExMort,totalBalIl,totalBcLimit,totalCuTl,totalIlHighCreditLimit,totalRevHiLim
0,0,0,2017-09-14T09:15:55.000-07:00,IL,606xx,51.1,52186.0,NaN,INDIVIDUAL,14999,...,0,89991,104947,19,89991,87632.0,6000,0,86247,18700
1,0,5,2017-09-21T23:04:20.000-07:00,NC,285xx,89.4,11683.0,NaN,INDIVIDUAL,4649,...,0,13948,15599,10,13948,13474.0,1000,0,13599,2000
2,0,1,2017-08-25T22:51:14.000-07:00,NY,112xx,79.3,47000.0,NaN,INDIVIDUAL,4772,...,0,23858,30100,9,23858,4331.0,19600,0,4500,25600
3,0,3,2017-08-26T14:52:52.000-07:00,FL,325xx,44.3,33800.0,117000.0,JOINT,2435,...,0,7304,16500,4,7304,0.0,15000,0,0,16500
4,0,10,2017-08-30T17:40:25.000-07:00,NY,146xx,52.4,24660.0,NaN,INDIVIDUAL,2907,...,0,31975,60967,16,31975,25598.0,16300,0,34467,26500


In [164]:
loans_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 184 columns):
accNowDelinq                  int64
accOpenPast24Mths             int64
acceptD                       object
addrState                     object
addrZip                       float64
allUtil                       float64
annualInc                     float64
annualIncJoint                float64
applicationType               object
avgCurBal                     int64
bcOpenToBuy                   float64
bcUtil                        float64
chargeoffWithin12Mths         int64
collections12MthsExMed        int64
creditPullD                   object
delinq2Yrs                    int64
delinqAmnt                    float64
desc                          object
dti                           float64
dtiJoint                      float64
earliestCrLine                object
empLength                     float64
empTitle                      object
expD                          object
ex

In [165]:
loans_df.id

0      119201096
1      119960755
2      116656675
3      115046312
4      118171263
5      119257410
6      118649689
7      119295486
8      118226827
9      119360834
10     119297582
11     119261704
12     119197842
13     119247308
14     119249601
15     119240315
16     118656299
17     119330980
18     118618418
19     119319773
20     119307675
21     119326603
22     119379037
23     119362471
24     118609828
25     119322239
26     119312436
27     118246557
28     119317763
29     119236109
         ...    
463    119703711
464    119397874
465    119386544
466    119381160
467    119249298
468    119331806
469    119407772
470    119388243
471    119387555
472    119413617
473    119334275
474    119698016
475    119374171
476    119960237
477    119313891
478    117880015
479    119716136
480    119271312
481    119342375
482    119377192
483    119260414
484    119695811
485    119475754
486    119367827
487    119380748
488    119384130
489    119362654
490    1197050

#### Convert EmpLength to years

In [132]:
loans_df.empLength = loans_df.empLength.apply(lambda x: x/60 if x >0 else 0)

#### Create month to int 1-12

In [133]:
loans_df['month'] = loans_df.acceptD.apply(lambda x: pd.to_datetime(x).month)

#### Crea cols term_36 and term_60

In [134]:
loans_df['term_36'] = [1 if i ==36 else 0 for i in loans_df.term]

In [135]:
loans_df['term_60'] = [1 if i ==60 else 0 for i in loans_df.term]

#### Create grade columns
- a:g

In [136]:
loans_df['grade_a'] = [1 if i =='A' else 0 for i in loans_df.grade]
loans_df['grade_b']= [1 if i =='B' else 0 for i in loans_df.grade]
loans_df['grade_c']= [1 if i =='C' else 0 for i in loans_df.grade]
loans_df['grade_d']= [1 if i =='D' else 0 for i in loans_df.grade]
loans_df['grade_e']= [1 if i =='E' else 0 for i in loans_df.grade]
loans_df['grade_f']= [1 if i =='F' else 0 for i in loans_df.grade]
loans_df['grade_g']= [1 if i =='G' else 0 for i in loans_df.grade]

### Create home ownership columns
-'home_ownership_ANY',
 'home_ownership_MORTGAGE',
 'home_ownership_NONE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',

In [137]:
loans_df['home_ownership_any'] = [1 if i =='ANY' else 0 for i in loans_df.homeOwnership]
loans_df['home_ownership_mortgage'] = [1 if i =='MORTGAGE' else 0 for i in loans_df.homeOwnership]
loans_df['home_ownership_none'] = [1 if i =='NONE' else 0 for i in loans_df.homeOwnership]
loans_df['home_ownership_other'] = [1 if i =='OTHER' else 0 for i in loans_df.homeOwnership]
loans_df['home_ownership_own'] = [1 if i =='OWN' else 0 for i in loans_df.homeOwnership]
loans_df['home_ownership_rent'] = [1 if i =='RENT' else 0 for i in loans_df.homeOwnership]

In [138]:
loans_df['home_ownership_any'].sum()

0

## Create purpose columns
-'purpose_credit_card',
 'purpose_debt_consolidation',
 'purpose_educational',
 'purpose_home_improvement',
 'purpose_house',
 'purpose_major_purchase',
 'purpose_medical',
 'purpose_moving',
 'purpose_other',
 'purpose_renewable_energy',
 'purpose_small_business',
 'purpose_vacation',
 'purpose_wedding',

In [139]:
loans_df['purpose_credit_card'] = [1 if i =='credit_card' else 0 for i in loans_df.purpose]
loans_df['purpose_debt_consolidation'] = [1 if i =='debt_consolidation' else 0 for i in loans_df.purpose]
loans_df['purpose_educational'] = [1 if i =='educational' else 0 for i in loans_df.purpose]
loans_df['purpose_home_improvement'] = [1 if i =='home_improvement' else 0 for i in loans_df.purpose]
loans_df['purpose_house'] = [1 if i =='house' else 0 for i in loans_df.purpose]
loans_df['purpose_major_purchase'] = [1 if i =='major_purchase' else 0 for i in loans_df.purpose]
loans_df['purpose_medical'] = [1 if i =='medical' else 0 for i in loans_df.purpose]
loans_df['purpose_moving'] = [1 if i =='moving' else 0 for i in loans_df.purpose]
loans_df['purpose_other'] = [1 if i =='other' else 0 for i in loans_df.purpose]
loans_df['purpose_renewable_energy'] = [1 if i =='renewable_energy' else 0 for i in loans_df.purpose]
loans_df['purpose_small_business'] = [1 if i =='small_business' else 0 for i in loans_df.purpose]
loans_df['purpose_vacation'] = [1 if i =='vacation' else 0 for i in loans_df.purpose]
loans_df['purpose_wedding'] = [1 if i =='wedding' else 0 for i in loans_df.purpose]
loans_df['purpose_car'] = [1 if i =='car' else 0 for i in loans_df.purpose]




### Create the states columns
'addr_state_AK',
 'addr_state_AL',
 'addr_state_AR',
 'addr_state_AZ',
 'addr_state_CA',
 'addr_state_CO',
 'addr_state_CT',
 'addr_state_DC',
 'addr_state_DE',
 'addr_state_FL',
 'addr_state_GA',
 'addr_state_HI',
 'addr_state_IA',
 'addr_state_ID',
 'addr_state_IL',
 'addr_state_IN',
 'addr_state_KS',
 'addr_state_KY',
 'addr_state_LA',
 'addr_state_MA',
 'addr_state_MD',
 'addr_state_ME',
 'addr_state_MI',
 'addr_state_MN',
 'addr_state_MO',
 'addr_state_MS',
 'addr_state_MT',
 'addr_state_NC',
 'addr_state_ND',
 'addr_state_NE',
 'addr_state_NH',
 'addr_state_NJ',
 'addr_state_NM',
 'addr_state_NV',
 'addr_state_NY',
 'addr_state_OH',
 'addr_state_OK',
 'addr_state_OR',
 'addr_state_PA',
 'addr_state_RI',
 'addr_state_SC',
 'addr_state_SD',
 'addr_state_TN',
 'addr_state_TX',
 'addr_state_UT',
 'addr_state_VA',
 'addr_state_VT',
 'addr_state_WA',
 'addr_state_WI',
 'addr_state_WV',
 'addr_state_WY'

In [140]:
loans_df['addr_state_AK'] = [1 if i =='AK' else 0 for i in loans_df.addrState]
loans_df['addr_state_AL'] = [1 if i =='AL' else 0 for i in loans_df.addrState]
loans_df['addr_state_AR'] = [1 if i =='AR' else 0 for i in loans_df.addrState]
loans_df['addr_state_AZ'] = [1 if i =='AZ' else 0 for i in loans_df.addrState]
loans_df['addr_state_CA'] = [1 if i =='CA' else 0 for i in loans_df.addrState]
loans_df['addr_state_CO'] = [1 if i =='CO' else 0 for i in loans_df.addrState]
loans_df['addr_state_CT'] = [1 if i =='CT' else 0 for i in loans_df.addrState]
loans_df['addr_state_DC'] = [1 if i =='DC' else 0 for i in loans_df.addrState]
loans_df['addr_state_DE'] = [1 if i =='DE' else 0 for i in loans_df.addrState]
loans_df['addr_state_FL'] = [1 if i =='FL' else 0 for i in loans_df.addrState]
loans_df['addr_state_GA'] = [1 if i =='GA' else 0 for i in loans_df.addrState]
loans_df['addr_state_HI'] = [1 if i =='HI' else 0 for i in loans_df.addrState]
loans_df['addr_state_IA'] = [1 if i =='IA' else 0 for i in loans_df.addrState]
loans_df['addr_state_ID'] = [1 if i =='ID' else 0 for i in loans_df.addrState]
loans_df['addr_state_IL'] = [1 if i =='IL' else 0 for i in loans_df.addrState]
loans_df['addr_state_IN'] = [1 if i =='IN' else 0 for i in loans_df.addrState]
loans_df['addr_state_KS'] = [1 if i =='KS' else 0 for i in loans_df.addrState]
loans_df['addr_state_KY'] = [1 if i =='KY' else 0 for i in loans_df.addrState]
loans_df['addr_state_LA'] = [1 if i =='LA' else 0 for i in loans_df.addrState]
loans_df['addr_state_MA'] = [1 if i =='MA' else 0 for i in loans_df.addrState]
loans_df['addr_state_MD'] = [1 if i =='MD' else 0 for i in loans_df.addrState]
loans_df['addr_state_ME'] = [1 if i =='ME' else 0 for i in loans_df.addrState]
loans_df['addr_state_MI'] = [1 if i =='MI' else 0 for i in loans_df.addrState]
loans_df['addr_state_MN'] = [1 if i =='MN' else 0 for i in loans_df.addrState]
loans_df['addr_state_MO'] = [1 if i =='MO' else 0 for i in loans_df.addrState]
loans_df['addr_state_MS'] = [1 if i =='MS' else 0 for i in loans_df.addrState]
loans_df['addr_state_MT'] = [1 if i =='MT' else 0 for i in loans_df.addrState]
loans_df['addr_state_NC'] = [1 if i =='NC' else 0 for i in loans_df.addrState]
loans_df['addr_state_ND'] = [1 if i =='ND' else 0 for i in loans_df.addrState]
loans_df['addr_state_NE'] = [1 if i =='NE' else 0 for i in loans_df.addrState]
loans_df['addr_state_NH'] = [1 if i =='NH' else 0 for i in loans_df.addrState]
loans_df['addr_state_NJ'] = [1 if i =='NJ' else 0 for i in loans_df.addrState]
loans_df['addr_state_NM'] = [1 if i =='NM' else 0 for i in loans_df.addrState]
loans_df['addr_state_NV'] = [1 if i =='NV' else 0 for i in loans_df.addrState]
loans_df['addr_state_NY'] = [1 if i =='NY' else 0 for i in loans_df.addrState]
loans_df['addr_state_OH'] = [1 if i =='OH' else 0 for i in loans_df.addrState]
loans_df['addr_state_OK'] = [1 if i =='OK' else 0 for i in loans_df.addrState]
loans_df['addr_state_OR'] = [1 if i =='OR' else 0 for i in loans_df.addrState]
loans_df['addr_state_PA'] = [1 if i =='PA' else 0 for i in loans_df.addrState]
loans_df['addr_state_RI'] = [1 if i =='RI' else 0 for i in loans_df.addrState]
loans_df['addr_state_SC'] = [1 if i =='SC' else 0 for i in loans_df.addrState]
loans_df['addr_state_SD'] = [1 if i =='SD' else 0 for i in loans_df.addrState]
loans_df['addr_state_TN'] = [1 if i =='TN' else 0 for i in loans_df.addrState]
loans_df['addr_state_TX'] = [1 if i =='TX' else 0 for i in loans_df.addrState]
loans_df['addr_state_UT'] = [1 if i =='UT' else 0 for i in loans_df.addrState]
loans_df['addr_state_VA'] = [1 if i =='VA' else 0 for i in loans_df.addrState]
loans_df['addr_state_VT'] = [1 if i =='VT' else 0 for i in loans_df.addrState]
loans_df['addr_state_WA'] = [1 if i =='WA' else 0 for i in loans_df.addrState]
loans_df['addr_state_WI'] = [1 if i =='WI' else 0 for i in loans_df.addrState]
loans_df['addr_state_WV'] = [1 if i =='WV' else 0 for i in loans_df.addrState]
loans_df['addr_state_WY'] = [1 if i =='WY' else 0 for i in loans_df.addrState]



#### Create the zip code column

In [141]:
int(loans_df.addrZip[0][:3])

606

- Drop null zip codes

In [142]:
loans_df.addrZip = loans_df.addrZip.apply( lambda x: float(x[:3]))

In [143]:
# Columns to match the data model
# empLength is in months - convert to years
api_cols =['loanAmount',
           'fundedAmount',
           'intRate',
           'installment',
           'empLength',
           'annualInc',
           'addrZip',
          'dti',
           'delinq2Yrs',
           'inqLast6Mths',
       'mthsSinceLastDelinq',
           'mthsSinceLastRecord',
           'openAcc',
           'pubRec',
           'revolBal',
           'revolUtil',
           'totalAcc',
      'collections12MthsExMed',
           'mthsSinceLastMajorDerog',
           'accNowDelinq',
           'totCollAmt',
           'totCurBal',
           'openAcc6m',
      'openIl6m',
        'openIl12m',
           'openIl24m',
           'totalBalIl',
           'iLUtil',
           'maxBalBc',
           'allUtil',
        'totalRevHiLim',
           'inqLast12m',
           'accOpenPast24Mths',
      'avgCurBal',
           'bcUtil',
           'chargeoffWithin12Mths',
           'delinqAmnt',
           'moSinOldIlAcct',
           'moSinOldRevTlOp',
           'moSinRcntRevTlOp',
     'moSinRcntTl',
           'mortAcc',
           'mthsSinceRecentBc',
           'mthsSinceRecentBcDlq',
           'mthsSinceRecentInq',
           'mthsSinceRecentRevolDelinq',
      'numAcctsEver120Ppd',
           'numActvBcTl',
           'numActvRevTl',
           'numBcSats',
           'numBcTl',
           'numIlTl',
           'numOpRevTl',
           'numRevAccts',
      'numRevTlBalGt0',
           'numSats',
           'numTl120dpd2m',
           'numTl30dpd',
           'numTl90gDpd24m',
           'numTlOpPast12m',
           'pctTlNvrDlq',
      'percentBcGt75',
           'pubRecBankruptcies',
           'taxLiens',
           'totHiCredLim',
           'totalBalExMort',
           'totalBcLimit',
           'totalIlHighCreditLimit',
      'month',
           'term_36',
           'term_60',
           'grade_a',
           'grade_b',
           'grade_c',
           'grade_d',
           'grade_e',
           'grade_f',
           'grade_g',
      'home_ownership_any','home_ownership_mortgage','home_ownership_none','home_ownership_other','home_ownership_own',
      'home_ownership_rent',
           'purpose_car','purpose_credit_card','purpose_debt_consolidation','purpose_educational','purpose_home_improvement',
      'purpose_house','purpose_major_purchase','purpose_medical','purpose_moving','purpose_other','purpose_renewable_energy',
      'purpose_small_business','purpose_vacation','purpose_wedding',
           'addr_state_AK','addr_state_AL','addr_state_AR',
      'addr_state_AZ','addr_state_CA','addr_state_CO','addr_state_CT','addr_state_DC','addr_state_DE','addr_state_FL',
      'addr_state_GA','addr_state_HI','addr_state_IA','addr_state_ID','addr_state_IL','addr_state_IN','addr_state_KS',
      'addr_state_KY','addr_state_LA','addr_state_MA','addr_state_MD','addr_state_ME','addr_state_MI','addr_state_MN',
      'addr_state_MO','addr_state_MS','addr_state_MT','addr_state_NC','addr_state_ND','addr_state_NE','addr_state_NH',
      'addr_state_NJ','addr_state_NM','addr_state_NV','addr_state_NY','addr_state_OH','addr_state_OK','addr_state_OR',
      'addr_state_PA','addr_state_RI','addr_state_SC','addr_state_SD','addr_state_TN','addr_state_TX','addr_state_UT',
      'addr_state_VA','addr_state_VT','addr_state_WA','addr_state_WI','addr_state_WV','addr_state_WY']



In [144]:
len(api_cols)

149

In [145]:
loans_df.columns[np.array([i  not in dm_cols for i in loans_df.columns])]

Index([          u'acceptD',         u'addrState',    u'annualIncJoint',
         u'applicationType',       u'bcOpenToBuy',       u'creditPullD',
                    u'desc',          u'dtiJoint',    u'earliestCrLine',
                u'empTitle',              u'expD',    u'expDefaultRate',
           u'ficoRangeHigh',      u'ficoRangeLow',             u'grade',
           u'homeOwnership',                u'id',           u'ilsExpD',
       u'initialListStatus',             u'inqFi',     u'investorCount',
                  u'isIncV',       u'isIncVJoint',             u'listD',
                u'memberId',   u'mthsSinceRcntIl',         u'openRv12m',
               u'openRv24m',           u'purpose',      u'reviewStatus',
           u'reviewStatusD',    u'serviceFeeRate',          u'subGrade',
               u'totalCuTl',       u'purpose_car',     u'addr_state_AR',
           u'addr_state_AZ'],
      dtype='object')

### Create the final data frame

In [148]:
final_api_df = loans_df[api_cols]

# Data Model
#### This is what the model was trained on
```'loan_amnt',
 'funded_amnt',
 'int_rate',
 'installment',
 'emp_length',
 'annual_inc',
 'zip_code',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 'open_il_6m',
 'open_il_12m',
 'open_il_24m',
 'total_bal_il',
 'il_util',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_last_12m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_util',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'mths_since_recent_bc',
 'mths_since_recent_bc_dlq',
 'mths_since_recent_inq',
 'mths_since_recent_revol_delinq',
 'num_accts_ever_120_pd',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_il_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_rev_tl_bal_gt_0',
 'num_sats',
 'num_tl_120dpd_2m',
 'num_tl_30dpd',
 'num_tl_90g_dpd_24m',
 'num_tl_op_past_12m',
 'pct_tl_nvr_dlq',
 'percent_bc_gt_75',
 'pub_rec_bankruptcies',
 'tax_liens',
 'tot_hi_cred_lim',
 'total_bal_ex_mort',
 'total_bc_limit',
 'total_il_high_credit_limit',
 'month',
 'term_ 36 months',
 'term_ 60 months',
 'grade_A',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'home_ownership_ANY',
 'home_ownership_MORTGAGE',
 'home_ownership_NONE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'purpose_car',
 'purpose_credit_card',
 'purpose_debt_consolidation',
 'purpose_educational',
 'purpose_home_improvement',
 'purpose_house',
 'purpose_major_purchase',
 'purpose_medical',
 'purpose_moving',
 'purpose_other',
 'purpose_renewable_energy',
 'purpose_small_business',
 'purpose_vacation',
 'purpose_wedding',
 'addr_state_AK',
 'addr_state_AL',
 'addr_state_AR',
 'addr_state_AZ',
 'addr_state_CA',
 'addr_state_CO',
 'addr_state_CT',
 'addr_state_DC',
 'addr_state_DE',
 'addr_state_FL',
 'addr_state_GA',
 'addr_state_HI',
 'addr_state_IA',
 'addr_state_ID',
 'addr_state_IL',
 'addr_state_IN',
 'addr_state_KS',
 'addr_state_KY',
 'addr_state_LA',
 'addr_state_MA',
 'addr_state_MD',
 'addr_state_ME',
 'addr_state_MI',
 'addr_state_MN',
 'addr_state_MO',
 'addr_state_MS',
 'addr_state_MT',
 'addr_state_NC',
 'addr_state_ND',
 'addr_state_NE',
 'addr_state_NH',
 'addr_state_NJ',
 'addr_state_NM',
 'addr_state_NV',
 'addr_state_NY',
 'addr_state_OH',
 'addr_state_OK',
 'addr_state_OR',
 'addr_state_PA',
 'addr_state_RI',
 'addr_state_SC',
 'addr_state_SD',
 'addr_state_TN',
 'addr_state_TX',
 'addr_state_UT',
 'addr_state_VA',
 'addr_state_VT',
 'addr_state_WA',
 'addr_state_WI',
 'addr_state_WV',
 'addr_state_WY'] ```

In [121]:
total_x_cols = ['loan_amnt',
 'funded_amnt',
 'int_rate',
 'installment',
 'emp_length',
 'annual_inc',
 'zip_code',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 'open_il_6m',
 'open_il_12m',
 'open_il_24m',
 'total_bal_il',
 'il_util',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_last_12m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_util',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'mths_since_recent_bc',
 'mths_since_recent_bc_dlq',
 'mths_since_recent_inq',
 'mths_since_recent_revol_delinq',
 'num_accts_ever_120_pd',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_il_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_rev_tl_bal_gt_0',
 'num_sats',
 'num_tl_120dpd_2m',
 'num_tl_30dpd',
 'num_tl_90g_dpd_24m',
 'num_tl_op_past_12m',
 'pct_tl_nvr_dlq',
 'percent_bc_gt_75',
 'pub_rec_bankruptcies',
 'tax_liens',
 'tot_hi_cred_lim',
 'total_bal_ex_mort',
 'total_bc_limit',
 'total_il_high_credit_limit',
 'month',
 'term_ 36 months',
 'term_ 60 months',
 'grade_A',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'home_ownership_ANY',
 'home_ownership_MORTGAGE',
 'home_ownership_NONE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'purpose_car',
 'purpose_credit_card',
 'purpose_debt_consolidation',
 'purpose_educational',
 'purpose_home_improvement',
 'purpose_house',
 'purpose_major_purchase',
 'purpose_medical',
 'purpose_moving',
 'purpose_other',
 'purpose_renewable_energy',
 'purpose_small_business',
 'purpose_vacation',
 'purpose_wedding',
 'addr_state_AK',
 'addr_state_AL',
 'addr_state_AR',
 'addr_state_AZ',
 'addr_state_CA',
 'addr_state_CO',
 'addr_state_CT',
 'addr_state_DC',
 'addr_state_DE',
 'addr_state_FL',
 'addr_state_GA',
 'addr_state_HI',
 'addr_state_IA',
 'addr_state_ID',
 'addr_state_IL',
 'addr_state_IN',
 'addr_state_KS',
 'addr_state_KY',
 'addr_state_LA',
 'addr_state_MA',
 'addr_state_MD',
 'addr_state_ME',
 'addr_state_MI',
 'addr_state_MN',
 'addr_state_MO',
 'addr_state_MS',
 'addr_state_MT',
 'addr_state_NC',
 'addr_state_ND',
 'addr_state_NE',
 'addr_state_NH',
 'addr_state_NJ',
 'addr_state_NM',
 'addr_state_NV',
 'addr_state_NY',
 'addr_state_OH',
 'addr_state_OK',
 'addr_state_OR',
 'addr_state_PA',
 'addr_state_RI',
 'addr_state_SC',
 'addr_state_SD',
 'addr_state_TN',
 'addr_state_TX',
 'addr_state_UT',
 'addr_state_VA',
 'addr_state_VT',
 'addr_state_WA',
 'addr_state_WI',
 'addr_state_WV',
 'addr_state_WY']

In [122]:
y_col = ['repaid']

In [123]:
len(total_x_cols)

149

In [124]:
len(api_cols)

149

- Same number of columns and in the correct order 


### Impute zeros for NaNs

In [156]:
final_api_df.fillna(0,inplace=True)

/usr/local/lib/python2.7/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [160]:
final_api_df.iloc[:,:20] .head()

,loanAmount,fundedAmount,intRate,installment,empLength,annualInc,addrZip,dti,delinq2Yrs,inqLast6Mths,mthsSinceLastDelinq,mthsSinceLastRecord,openAcc,pubRec,revolBal,revolUtil,totalAcc,collections12MthsExMed,mthsSinceLastMajorDerog,accNowDelinq
0,5000.0,4975.0,7.21,154.87,1.0,52186.0,606.0,2.21,0,0,31.0,0.0,7,0,2359.0,12.6,19,0,0.0,0
1,3500.0,3475.0,30.65,149.83,0.4,11683.0,285.0,15.01,0,0,0.0,0.0,3,0,474.0,23.7,10,0,0.0,0
2,3000.0,2725.0,14.08,102.65,0.0,47000.0,112.0,20.63,0,1,0.0,0.0,5,0,19527.0,76.3,9,0,0.0,0
3,15000.0,10350.0,16.02,527.51,0.0,33800.0,325.0,8.63,0,1,0.0,0.0,3,0,7304.0,44.3,4,0,0.0,0
4,8000.0,3300.0,21.45,303.26,0.6,24660.0,146.0,38.44,0,1,43.0,0.0,13,0,6377.0,24.1,16,0,0.0,0


In [161]:
final_api_df.iloc[:,20:40] .head()

,totCollAmt,totCurBal,openAcc6m,openIl6m,openIl12m,openIl24m,totalBalIl,iLUtil,maxBalBc,allUtil,totalRevHiLim,inqLast12m,accOpenPast24Mths,avgCurBal,bcUtil,chargeoffWithin12Mths,delinqAmnt,moSinOldIlAcct,moSinOldRevTlOp,moSinRcntRevTlOp
0,0,89991,0,2,0,0,87632.0,83.9,2332.0,51.1,18700,0,0,14999,38.9,0,0.0,148.0,106,26
1,0,13948,1,1,3,3,13474.0,99.1,474.0,89.4,2000,1,5,4649,47.4,0,0.0,66.0,23,22
2,0,23858,1,1,1,1,4331.0,96.2,9506.0,79.3,25600,1,1,4772,99.6,0,0.0,89.0,406,86
3,0,7304,2,0,0,0,0.0,0.0,4926.0,44.3,16500,1,3,2435,48.2,0,0.0,0.0,67,3
4,0,31975,1,2,0,1,25598.0,74.3,1368.0,52.4,26500,3,10,2907,26.0,0,0.0,27.0,61,3


# Create orders
```{
	"aid":11111,
	"orders":[
	{
		"loanId":22222,
		"requestedAmount":55.0,
		"portfolioId":44444
	},
	{
		"loanId": 33333,
		"requestedAmount":25,
		"portfolioId":55555
	},
	{
		"loanId": 44444,
		"requestedAmount":25,
	}]
}```

In [167]:
loan_ids = loans_df.id